# [자연어처리]
# 4주차(4-1). BOW 기반의 문서 분류(2)


# 202002961 김현주

# 이제 Colab 사용에 모든 수강생분들이 실행에는 어느정도 익숙해 졌을 거라 생각됩니다!
* (벌써 4주차 입니다. 시간 빠르죠?)
* **키보드**(+마우스)를 조금 더 사용해보도록 하겠습니다.
> ## 중간 중간 **'### ... 이 부분을 완성하시오'** 라는 부분의 코드를 완성해 보기 바랍니다. (강의 자료를 참고하여 '???' 부분을 작성해 보시기 바랍니다.)





## 3-2. 실습 리뷰 시작
## 20 뉴스그룹 데이터 준비 및 특성 추출

 http://scikit-learn.org/0.19/datasets/twenty_newsgroups.html

In [1]:
from sklearn.datasets import fetch_20newsgroups

# 20개의 토픽 중 선택하고자 하는 토픽을 리스트로 생성
categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']

# 학습 데이터셋을 가져옴
# 메일 내용에서 hint가 되는 부분을 삭제 - 순수하게 내용만으로 분류
newsgroups_train = fetch_20newsgroups(subset='train',
                                      remove=('headers', 'footers', 'quotes'),
                                      categories=categories)
# 테스트 데이터셋을 가져옴
newsgroups_test = fetch_20newsgroups(subset='test',
                                     remove=('headers', 'footers', 'quotes'),
                                     categories=categories)

print('#Train set size:', len(newsgroups_train.data))
print('#Test set size:', len(newsgroups_test.data))
print('#Selected categories:', newsgroups_train.target_names)
print('#Train labels:', set(newsgroups_train.target))

#Train set size: 2034
#Test set size: 1353
#Selected categories: ['alt.atheism', 'comp.graphics', 'sci.space', 'talk.religion.misc']
#Train labels: {0, 1, 2, 3}


In [2]:
X_train = newsgroups_train.data   # 학습 데이터셋 문서
y_train = newsgroups_train.target # 학습 데이터셋 레이블

X_test = newsgroups_test.data     # 테스트 데이터셋 문서
y_test = newsgroups_test.target   # 테스트 데이터셋 레이블

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF 방식 / feature의 개수는 2000개를 사용
tfidf = TfidfVectorizer(max_features=2000, min_df=5, max_df=0.5)

# train set을 변환
X_train_tfidf = tfidf.fit_transform(X_train)
# test set을 변환
X_test_tfidf = tfidf.transform(X_test)

In [4]:
# sklearn이 제공하는 MultinomialNB 를 사용
from sklearn.naive_bayes import MultinomialNB

# 분류기 선언
NB_clf = MultinomialNB()
# tfidf train set을 이용하여 분류기(classifier)를 새로 학습
NB_clf.fit(X_train_tfidf, y_train)

print('Train set score: {:.3f}'.format(NB_clf.score(X_train_tfidf, y_train))) # train set에 대한 예측정확도를 확인
print('Test set score: {:.3f}'.format(NB_clf.score(X_test_tfidf, y_test))) # test set에 대한 예측정확도를 확인

Train set score: 0.862
Test set score: 0.741


# 3-2. 실습 리뷰 끝
---
# 4-1. 수업 내용 시작



## 4. 로지스틱 회귀분석을 이용한 문서 분류

1: http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

In [5]:
# sklearn이 제공하는 LogisticRegression을 사용
from sklearn.linear_model import LogisticRegression

# count vector에 대해 regression을 해서 NB와 비교
LR_clf = LogisticRegression() # 분류기 선언
LR_clf.fit(X_train_tfidf, y_train) # train data를 이용하여 분류기를 학습

print('Train set score: {:.3f}'.format(LR_clf.score(X_train_tfidf, y_train))) # train data에 대한 예측정확도
print('Test set score: {:.3f}'.format(LR_clf.score(X_test_tfidf, y_test))) # test data에 대한 예측정확도

Train set score: 0.929
Test set score: 0.735


In [6]:
LR_clf.coef_ # 계수 확인

array([[-0.29017615,  0.1789959 , -0.19273631, ..., -0.40956462,
        -0.08686455, -0.11764324],
       [ 0.42032388, -0.15751563,  0.52745772, ..., -0.13140309,
        -0.05589633,  0.27076447],
       [ 0.10923797,  0.10729818, -0.17887524, ..., -0.2709849 ,
         0.10903719, -0.06967447],
       [-0.2393857 , -0.12877846, -0.15584617, ...,  0.81195261,
         0.03372369, -0.08344677]])

In [7]:
len(-LR_clf.coef_[0])

2000

In [8]:
import numpy as np

top10 = np.argsort(-LR_clf.coef_[0])[:10]
top10

array([ 214, 1486,  281,  216,  913,  914,  498,  215, 1153, 1424])

### top10_features 함수 정의
* 텍스트 분류 모델이 **각 카테고리에 가장 영향을 많이 미치는 상위 10개의 특성(feature)**을 출력

In [9]:
def top10_features(classifier, vectorizer, categories):
    feature_names = np.asarray(vectorizer.get_feature_names_out())
    for i, category in enumerate(categories):
        # 역순으로 정렬하기 위해 계수에 음수를 취해서 정렬 후 앞에서부터 10개의 값을 반환
        top10 = np.argsort(-classifier.coef_[i])[:10]
        # 카테고리와 영향이 큰 특성 10개를 출력
        print("%s: %s" % (category, ", ".join(feature_names[top10])))

In [10]:
top10_features(LR_clf, tfidf, newsgroups_train.target_names)

alt.atheism: atheism, religion, bobby, atheists, islam, islamic, deletion, atheist, motto, punishment
comp.graphics: graphics, image, file, computer, 3d, files, hi, looking, points, code
sci.space: space, nasa, orbit, launch, moon, spacecraft, shuttle, earth, lunar, flight
talk.religion.misc: christian, christians, jesus, god, fbi, objective, order, his, he, blood


### 라쏘 회귀분석(lasso regression)을 이용한 특성 선택(feature selection)

1. http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html

In [11]:
lasso_clf = LogisticRegression(penalty='l1', solver='liblinear', C=1) # Lasso는 동일한 LogisticRegression을 사용하면서 매개변수로 지정
lasso_clf.fit(X_train_tfidf, y_train) # train data로 학습

print('#Train set score: {:.3f}'.format(lasso_clf.score(X_train_tfidf, y_train)))
print('#Test set score: {:.3f}'.format(lasso_clf.score(X_test_tfidf, y_test)))

#Train set score: 0.819
#Test set score: 0.724


In [12]:
# 계수(coefficient) 중에서 0이 아닌 것들의 개수를 출력
print('#Used features count: {}'.format(np.sum(lasso_clf.coef_ != 0)), 'out of', X_train_tfidf.shape[1])

#Used features count: 437 out of 2000


In [13]:
top10_features(lasso_clf, tfidf, newsgroups_train.target_names)

alt.atheism: bobby, atheism, atheists, islam, religion, islamic, motto, atheist, satan, vice
comp.graphics: graphics, image, 3d, file, computer, hi, video, files, looking, sphere
sci.space: space, orbit, launch, nasa, spacecraft, flight, moon, dc, shuttle, solar
talk.religion.misc: fbi, christian, christians, christ, order, jesus, children, objective, context, blood


### 릿지 회귀(ridge regression)


In [14]:
from sklearn.linear_model import RidgeClassifier

ridge_clf = RidgeClassifier() # 릿지 분류기 선언
ridge_clf.fit(X_train_tfidf, y_train) # train data로 학습

print('Train set score: {:.3f}'.format(ridge_clf.score(X_train_tfidf, y_train)))
print('Test set score: {:.3f}'.format(ridge_clf.score(X_test_tfidf, y_test)))

Train set score: 0.960
Test set score: 0.735


In [15]:
top10_features(ridge_clf, tfidf, newsgroups_train.target_names)

alt.atheism: bobby, religion, atheists, atheism, motto, punishment, satan, deletion, islamic, liar
comp.graphics: graphics, computer, 3d, file, 42, hi, image, using, screen, looking
sci.space: space, orbit, spacecraft, sci, moon, funding, nasa, 23, engineering, nick
talk.religion.misc: christian, blood, christians, fbi, order, objective, hudson, children, abortion, dead


In [16]:
# alpha 값을 0.1로 할 경우
ridge_clf = RidgeClassifier(alpha=0.1) # 릿지 분류기 선언
ridge_clf.fit(X_train_tfidf, y_train) # 학습

print('Train set score: {:.3f}'.format(ridge_clf.score(X_train_tfidf, y_train)))
print('Test set score: {:.3f}'.format(ridge_clf.score(X_test_tfidf, y_test)))

Train set score: 0.977
Test set score: 0.673


scikit-learn 라이브러리의 train_test_split 함수를 사용하여, 데이터를 학습용(train)과 검증용(validation) 데이터로 나누는 작업 수행

In [17]:
from sklearn.model_selection import train_test_split

# 8:2의 비율로 나누기
X_train_ridge, X_val_ridge, y_train_ridge, y_val_ridge = train_test_split(X_train_tfidf, y_train, test_size=0.2, random_state=42)

alpha 값을 0.1부터 10까지 0.1씩 증가시키며, 가장 적절한 alpha 값 찾기
* 가장 적절한 alpha 값? 검증 데이터(validation)에 대해서 가장 좋은 성능을 보이는 alpha 값

In [18]:
max_score = 0
max_alpha = 0
for alpha in np.arange(0.1, 10, 0.1): # alpha를 0.1부터 10까지 0.1씩 증가
    ridge_clf = RidgeClassifier(alpha=alpha) # 릿지 분류기 선언
    ridge_clf.fit(X_train_ridge, y_train_ridge) # 학습
    score = ridge_clf.score(X_val_ridge, y_val_ridge) # 검정 데이터셋에 대해 정확도를 측정
    if score > max_score: # 정확도가 이전의 정확도 최대값보다 크면 최대값을 변경한다.
        max_score = score
        max_alpha = alpha
print('Max alpha {:.3f} at max validation score {:.3f}'.format(max_alpha, max_score))

Max alpha 1.600 at max validation score 0.826


검증 데이터에 대해, 0.826의 성능을 보이는 alpha=1.6이 가장 적절한 값으로 나옴

In [19]:
# 릿지 분류기 선언
ridge_clf = RidgeClassifier(alpha=1.6)
ridge_clf.fit(X_train_tfidf, y_train) # 학습

print('Train set score: {:.3f}'.format(ridge_clf.score(X_train_tfidf, y_train)))
print('Test set score: {:.3f}'.format(ridge_clf.score(X_test_tfidf, y_test)))

Train set score: 0.947
Test set score: 0.739


In [20]:
top10_features(ridge_clf, tfidf, newsgroups_train.target_names)

alt.atheism: bobby, religion, atheism, atheists, motto, punishment, islam, deletion, islamic, satan
comp.graphics: graphics, computer, 3d, file, image, hi, 42, using, screen, looking
sci.space: space, orbit, nasa, spacecraft, moon, sci, launch, flight, funding, idea
talk.religion.misc: christian, christians, fbi, blood, order, jesus, objective, children, christ, hudson


## 5. 결정트리 등을 이용한 기타 문서 분류 방법

1. https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html
2. https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
3. https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html

In [21]:
# DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(random_state=7)
tree.fit(X_train_tfidf, y_train)

print('#Decision Tree train set score: {:.3f}'.format(tree.score(X_train_tfidf, y_train)))
print('#Decision Tree test set score: {:.3f}'.format(tree.score(X_test_tfidf, y_test)))

#Decision Tree train set score: 0.977
#Decision Tree test set score: 0.536


In [22]:
# RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(random_state=7)
forest.fit(X_train_tfidf, y_train)
print('#Random Forest train set score: {:.3f}'.format(forest.score(X_train_tfidf, y_train)))
print('#Random Forest test set score: {:.3f}'.format(forest.score(X_test_tfidf, y_test)))

#Random Forest train set score: 0.977
#Random Forest test set score: 0.685


In [23]:
# GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=7)
gb.fit(X_train_tfidf, y_train)
print('#Gradient Boosting train set score: {:.3f}'.format(gb.score(X_train_tfidf, y_train)))
print('#Gradient Boosting test set score: {:.3f}'.format(gb.score(X_test_tfidf, y_test)))

#Gradient Boosting train set score: 0.933
#Gradient Boosting test set score: 0.695


In [24]:
gb.feature_importances_

array([0.        , 0.        , 0.00018793, ..., 0.00321606, 0.        ,
       0.        ])

In [25]:
sorted(gb.feature_importances_, reverse=True)[:40]

[0.12606815556503395,
 0.07955738535484685,
 0.023540357925013154,
 0.02306185042035626,
 0.020555812402388306,
 0.020063273938036058,
 0.018092916558127174,
 0.01770949154089047,
 0.016857094927865728,
 0.015308260768876486,
 0.015023452519852189,
 0.013797145686816005,
 0.010426583770790024,
 0.010376173054679774,
 0.010225361352506715,
 0.010061136447910918,
 0.00973361829291745,
 0.0096630393064523,
 0.009402038359825497,
 0.008981046753380314,
 0.008827368636033596,
 0.007819020701620829,
 0.007809482123037147,
 0.007597096131104975,
 0.007260184843430255,
 0.00725511899882258,
 0.007074189152295386,
 0.006899060277543801,
 0.006698870635101266,
 0.006529364772733111,
 0.006245749237205096,
 0.005513872126424431,
 0.004918770389689076,
 0.004803787605613806,
 0.00476794021326829,
 0.004624503816321027,
 0.004608372640219639,
 0.004606426911938473,
 0.004415620817426229,
 0.004334578233902472]

In [26]:
# 분류에 크게 기여한 top40 단어들을 반환
sorted_feature_importances = sorted(zip(tfidf.get_feature_names_out(), gb.feature_importances_), key=lambda x: x[1], reverse=True)
for feature, value in sorted_feature_importances[:40]:
    print('%s: %.3f' % (feature, value), end=', ')

space: 0.126, graphics: 0.080, atheism: 0.024, thanks: 0.023, file: 0.021, orbit: 0.020, jesus: 0.018, god: 0.018, hi: 0.017, nasa: 0.015, image: 0.015, files: 0.014, christ: 0.010, moon: 0.010, bobby: 0.010, launch: 0.010, looking: 0.010, christian: 0.010, atheists: 0.009, christians: 0.009, fbi: 0.009, 3d: 0.008, you: 0.008, not: 0.008, religion: 0.007, islamic: 0.007, spacecraft: 0.007, flight: 0.007, computer: 0.007, islam: 0.007, ftp: 0.006, color: 0.006, software: 0.005, atheist: 0.005, card: 0.005, people: 0.005, koresh: 0.005, his: 0.005, kent: 0.004, sphere: 0.004, 

# **[[실습 4-1-1]]**
## 지난 시간(3-2)과 이번 시간(4-1)에 배운 분류기들의 train/test score를 비교해 보시오.
* 코드 실행 결과로 얻은 score 결과를 복사/붙여넣기 혹은 직접 입력
* 비교를 통해 어떤 분류기의 성능이 가장 좋은지 평가해 보시오.


# (해당 셀을 더블 클릭하여 수정)
1. 나이브 베이즈 분류기(Naive Bayse Classifier) - Train set score: 0.824, Test set score: 0.732
2. 로지스틱 회귀분석(LogisticRegression) - Train set score: 0.929, Test set score: 0.735
3. 라쏘 회귀분석(lasso regression) - Train set score: 0.819, Test set score: 0.724
4. 릿지 회귀(ridge regression) - Train set score: 0.960, Test set score: 0.735
5. 결정 트리(Decision Tree) - train set score: 0.977, test set score: 0.536
6. 랜덤 포레스트(Random Forest) - train set score: 0.977, test set score: 0.685
7. 그래디언트 부스팅(Gradient Boosting) - train set score: 0.933, test set score: 0.695

(내용 작성) 어떤 분류기의 성능이 가장 좋은가?

로지스틱 회귀와 릿지 회귀가 테스트 세트에서 동일한 정확도를 보이지만, 로지스틱 회귀는 훈련 세트와 테스트 세트 간의 성능 차이가 크지 않아 과적합에 대한 우려가 적습니다. 따라서 로지스틱 회귀의 성능이 가장 좋다고 생각합니다.

# **[[실습 4-1-2]]**
# coef_의 결과와 feature_importances_ 결과 비교를 통해, 회귀분석 방법과 트리 기반의 분류 방식의 차이점 알아보기
* coef_의 결과(여러 분류기 중 1개 이상)와 feature_importances_ 결과(여러 분류기 중 1개 이상) 복사/붙여넣기
* 비교를 통해 어떤 차이점이 있는 것 같은지 분석해보시오.

# (해당 셀을 더블 클릭하여 수정)

* coef_의 결과 : 로지스틱 회귀분석(LogisticRegression)

array([[-0.29017615,  0.1789959 , -0.19273631, ..., -0.40956462,
        -0.08686455, -0.11764324],
       [ 0.42032388, -0.15751563,  0.52745772, ..., -0.13140309,
        -0.05589633,  0.27076447],
       [ 0.10923797,  0.10729818, -0.17887524, ..., -0.2709849 ,
         0.10903719, -0.06967447],
       [-0.2393857 , -0.12877846, -0.15584617, ...,  0.81195261,
         0.03372369, -0.08344677]])

* feature_importances_ 결과 : 그래디언트 부스팅(Gradient Boosting)

array([0.        , 0.        , 0.00018793, ..., 0.00321606, 0.        ,
       0.        ])

(내용 작성) 회귀분석 방법과 트리 기반의 분류 방식은 어떤 차이점이 있는 것 같은가?

회귀분석 방법은 선형 모델이며, coef_는 각 문서에 대해서 각각의 단어들이 기여한 정도를 반환하는데, 이때 가중치의 부호와 크기를 통해 해당 단어가 분류 결과에 긍정적 또는 부정적 영향을 미치는지 명확히 파악할 수 있고 각 단어가 예측에 어떻게 기여하는지 해석이 용이합니다.

트리 기반의 분류 방식은 비선형 모델이며, feature_importances_는 각각의 단어들이 분류에 기여한 정도를 반환하는데, 트리 기반의 분류 방식은 비선형 모델이며, feature_importances_는 각각의 단어들이 분류에 기여한 정도를 반환하는데, 이 값은 모델이 의사결정 과정에서 해당 특성을 얼마나 자주 사용했는지를 나타내지만 각 특성이 결과에 미치는 영향을 직접적으로 설명할 수 없기 때문에 해석이 상대적으로 어렵습니다.

---

# [파일] -> [다운로드] -> [.ipynb 다운로드]



# 4-1. 수업 내용 끝
---